# Assignment 3

Welcome to Assignment 3. This will be even more fun. Now we will calculate statistical measures on the test data you have created.

YOU ARE NOT ALLOWED TO USE ANY OTHER 3RD PARTY LIBRARIES LIKE PANDAS. PLEASE ONLY MODIFY CONTENT INSIDE THE FUNCTION SKELETONS
Please read why: https://www.coursera.org/learn/exploring-visualizing-iot-data/discussions/weeks/3/threads/skjCbNgeEeapeQ5W6suLkA
. Just make sure you hit the play button on each cell from top to down. There are seven functions you have to implement. Please also make sure than on each change on a function you hit the play button again on the corresponding cell to make it available to the rest of this notebook.
Please also make sure to only implement the function bodies and DON'T add any additional code outside functions since this might confuse the autograder.

So the function below is used to make it easy for you to create a data frame from a cloudant data frame using the so called "DataSource" which is some sort of a plugin which allows ApacheSpark to use different data sources.


In [1]:
#Please don't modify this function
def readDataFrameFromCloudant(database):
    cloudantdata=spark.read.load(database, "org.apache.bahir.cloudant")

    cloudantdata.createOrReplaceTempView("washing")
    spark.sql("SELECT * from washing").show()
    return cloudantdata

All functions can be implemented using DataFrames, ApacheSparkSQL or RDDs. We are only interested in the result. You are given the reference to the data frame in the "df" parameter and in case you want to use SQL just use the "spark" parameter which is a reference to the global SparkSession object. Finally if you want to use RDDs just use "df.rdd" for obtaining a reference to the underlying RDD object. 

Let's start with the first function. Please calculate the minimal temperature for the test data set you have created. We've provided a little skeleton for you in case you want to use SQL. You can use this skeleton for all subsequent functions. Everything can be implemented using SQL only if you like.

In [4]:
def minTemperature(df,spark):
    return spark.sql("SELECT min(temperature) as mintemp from washing").first().mintemp

Please now do the same for the mean of the temperature

In [5]:
def meanTemperature(df,spark):
    return spark.sql("SELECT avg(temperature) as meantemp from washing").first().meantemp

Please now do the same for the maximum of the temperature

In [6]:
def maxTemperature(df,spark):
    return spark.sql("SELECT max(temperature) as maxtemp from washing").first().maxtemp

Please now do the same for the standard deviation of the temperature

In [12]:
def sdTemperature(df,spark):
    return spark.sql("SELECT stddev(temperature) as stddevtemp from washing").first().stddevtemp

Please now do the same for the skew of the temperature. Since the SQL statement for this is a bit more complicated we've provided a skeleton for you. You have to insert custom code at four position in order to make the function work. Alternatively you can also remove everything and implement if on your own. Note that we are making use of two previously defined functions, so please make sure they are correct. Also note that we are making use of python's string formatting capabilitis where the results of the two function calls to "meanTemperature" and "sdTemperature" are inserted at the "%s" symbols in the SQL string.

In [14]:
def skewTemperature(df,spark):    
    return spark.sql("""
SELECT 
    (
        1/count(temperature)
    ) *
    SUM (
        POWER(temperature-%s,3)/POWER(%s,3)
    )

as skewTemp from washing
                    """ %(meanTemperature(df,spark),sdTemperature(df,spark))).first().skewTemp

Kurtosis is the 4th statistical moment, so if you are smart you can make use of the code for skew which is the 3rd statistical moment. Actually only two things are different.

In [16]:
def kurtosisTemperature(df,spark):    
    return spark.sql("""
SELECT 
    (
        1/count(temperature)
    ) *
    SUM (
        POWER(temperature-%s,4)/POWER(%s,4)
    )

as kurtTemp from washing
                    """ %(meanTemperature(df,spark),sdTemperature(df,spark))).first().kurtTemp

Just a hint. This can be solved easily using SQL as well, but as shown in the lecture also using RDDs.

In [20]:
def correlationTemperatureHardness(df,spark):
    return df.stat.corr("temperature", "hardness")

### PLEASE DON'T REMOVE THIS BLOCK - THE FOLLOWING CODE IS NOT GRADED
#axx
### PLEASE DON'T REMOVE THIS BLOCK - THE FOLLOWING CODE IS NOT GRADED

Now it is time to connect to the cloudant database. Please have a look at the Video "Overview of end-to-end scenario" of Week 2 starting from 6:40 in order to learn how to obtain the credentials for the database. Please paste this credentials as strings into the below code

### TODO Please provide your Cloudant credentials here

In [2]:
hostname = "55a963c3-7df2-4855-8ff4-e79305986386-bluemix.cloudant.com"
user = "55a963c3-7df2-4855-8ff4-e79305986386-bluemix"
pw = "37092b4a08acfa5f76f64ae8fefd73df5ac9818d5bbae227cdda23e614529fb0"
database = "washing" #as long as you didn't change this in the NodeRED flow the database name stays the same

In [3]:
spark = SparkSession\
    .builder\
    .appName("Cloudant Spark SQL Example in Python using temp tables")\
    .config("cloudant.host",hostname)\
    .config("cloudant.username", user)\
    .config("cloudant.password",pw)\
    .getOrCreate()
cloudantdata=readDataFrameFromCloudant(database)

+--------------------+--------------------+-----+--------+----------+---------+--------+-----+-----------+-------------+-------+
|                 _id|                _rev|count|flowrate|fluidlevel|frequency|hardness|speed|temperature|           ts|voltage|
+--------------------+--------------------+-----+--------+----------+---------+--------+-----+-----------+-------------+-------+
|11f0ed8f54d498fd1...|1-d650f234ef1eab3...|    1|    null|      null|       67|    null| null|       null|1531831609230|    236|
|11f0ed8f54d498fd1...|1-a42e2e17a3ad85b...|    4|      11|acceptable|     null|      73| null|         88|1531831610246|   null|
|11f0ed8f54d498fd1...|1-eb4c9a97655cf95...|    7|      11|acceptable|     null|      77| null|         96|1531831613251|   null|
|11f0ed8f54d498fd1...|1-77aa413dd3d695e...|   13|      11|acceptable|     null|      80| null|         97|1531831619274|   null|
|11f0ed8f54d498fd1...|1-ffb0149c24e12f3...|   14|      11|acceptable|     null|      71| null|   

In [8]:
minTemperature(cloudantdata,spark)

80

In [9]:
meanTemperature(cloudantdata,spark)

90.66386554621849

In [10]:
maxTemperature(cloudantdata,spark)

100

In [13]:
sdTemperature(cloudantdata,spark)

6.113720760804424

In [15]:
skewTemperature(cloudantdata,spark)

-0.10560874355400261

In [17]:
kurtosisTemperature(cloudantdata,spark)

1.768216204755885

In [21]:
correlationTemperatureHardness(cloudantdata,spark)

0.9914930919353597

Congratulations, you are done, please download this notebook as python file using the export function and submit is to the gader using the filename "assignment3.1.py"